In [1]:
import time
import os
import requests
from tqdm import tqdm

from zhihu_oauth import ZhihuClient
from zhihu_oauth.exception import NeedCaptchaException

from bs4 import BeautifulSoup

s = requests.session()
s.keep_alive = False

In [2]:
# login ZhihuClient
client = ZhihuClient()
TOKEN_FILE = 'token.pkl'

if os.path.isfile(TOKEN_FILE):
    client.load_token(TOKEN_FILE)
else:
    client.login_in_terminal()
    client.save_token(TOKEN_FILE)

# function

In [3]:
def save_answer(topic, answer_numbers=0, save_path = 'zhihu'):
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    topic_path = save_path + '/' + topic.name
    if not os.path.exists(topic_path):
        os.mkdir(topic_path)

    for i,question in zip(tqdm(range(topic.questions_count)),topic.unanswered_questions):
        question_time = time.localtime(question.created_time)
        question_dt = time.strftime("%Y-%m-%d",question_time)
        # 时间筛选
        # if question_dt > '2019-12-31' or question_dt < '2019-11-23':
        #     continue
        question_name =  question_dt + '-' + question.title
        question_name = question_name.replace('/',' ')
        question_file = topic_path+'/'+question_name+'.txt'

        if os.path.exists(question_file):
            # 同一个问题不处理
            continue

        with open(question_file, 'w') as f:
            # question
            f.write('question is : ')
            f.write('\n')
            f.write(str(question.title))
            f.write('\n')
            f.write('\n')
            
            # answers
            for answer in question.answers:
                # 时间
                answer_time = time.localtime(answer.created_time)
                answer_year = answer_time.tm_year
                answer_dt = time.strftime("%Y-%m-%d %H:%M:%S",answer_time)
                if answer_year < 2017:
                    continue

                # 文本内容
                soup = BeautifulSoup(answer.content,'html.parser')
                text = soup.find_all(text=True)
                answer_name = answer_dt + '-' + answer.author.name

                # 评论内容
                comments = []
                for comment in answer.comments:
                    for reply in comment.replies:
                        comments.append(reply.content)

                # 保存txt文档
                # answer_file = question_path + '/' + answer_name + '.txt'

                # answer
                f.write('answer of {} is : '.format(answer.author.name))
                f.write('\n')
                for line in text:
                    f.write(line)
                    f.write('\n')

                # comments
                if comments != []:
                    f.write('comments of this answer: ')
                    f.write('\n')
                    for line in comments:
                        f.write(line)
                        f.write('\n')

                # voteup count
                f.write('voteup count: ' + str(answer.voteup_count))
                f.write('\n')

                # create time
                f.write('create_time: ' + answer_dt)
                f.write('\n')
                f.write('\n')

                answer_numbers += 1
    return answer_numbers

# Main topic

In [4]:
answer_numbers_all = 0
topic_id = 19649350  # 电子烟的topic id
topic = client.topic(topic_id)
topic_children = topic.children

answer_numbers_all += save_answer(topic, answer_numbers_all)
print('answer_numbers_all: ', answer_numbers_all)
for topic_child in topic_children:
    answer_numbers_all += save_answer(topic_child, answer_numbers_all)
    print('answer_numbers_all: ', answer_numbers_all)

print('answer numbers :',answer_numbers_all)

100%|██████████| 3869/3869 [08:29<00:00,  7.59it/s]  


answer_numbers_all:  711


  0%|          | 0/207 [00:00<?, ?it/s]

answer_numbers_all:  1422


  0%|          | 0/6 [00:00<?, ?it/s]

...

answer_numbers_all:  23291
answer_numbers_all:  46582




100%|██████████| 26/26 [00:24<00:00,  1.06it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

answer_numbers_all:  93204



100%|██████████| 99/99 [00:31<00:00,  3.10it/s]

answer_numbers_all:  186465
answer numbers : 186465
